In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as split

from keras.layers import LSTM, Input, Dense
from keras.models import Model

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli
from bitstring import BitArray

Using TensorFlow backend.


In [2]:
np.random.seed(1120)

In [3]:
data = pd.read_csv('./data/AT.dat', delimiter="\t",
                   skiprows=[0], names=["k","r","x","A[k,r,x]"])
print(data.head())
data = data.to_numpy()

# Use first 8,056 points as training/validation and rest as test set.
train_data = data[0:8056]
test_data = data[8056:]

       k         r         x  A[k,r,x]
0  0.001  0.010817 -0.973907  1.009217
1  0.001  0.010817 -0.679410  1.009143
2  0.001  0.010817 -0.148874  1.009077
3  0.001  0.010817  0.148874  1.009077
4  0.001  0.010817  0.679410  1.009142


### Implementation
Now, we have a fair understanding of what GA is and how it works. Next, let’s get to coding.

We will use wind power forecast data, which is available at the following link. It consists of normalized (between zero and one) wind power measurements from seven wind farms. To keep things simple, we will use first wind farm data (column named wp1) but I encourage the reader to experiment and extend the code to forecast energy for all seven, wind farms.

Let’s import required packages, load the dataset and define two helper functions. The first method prepare_dataset will segment the data into chunks to create X, Y pair for model training. The X will the wind power values from the past (e.g. 1 to t-1) and Y will be future value at time t. The second method train_evaluate perform three things, 1) decoding GA solution to get window size and number of units. 2) Prepare the dataset using window size found by GA and divide into train and validation set, and 3) train LSTM model, calculate RMSE on validation set and return it as a fitness score of the current GA solution.

In [4]:
data[:,1].shape
data[:,0:3].shape

(8556, 3)

In [5]:
np.zeros(11)[6:11]

array([0., 0., 0., 0., 0.])

In [15]:
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:,0:3]
    Y = data[:,3]  
    return X, Y

def train_evaluate(ga_individual_solution):   
    # Decode GA solution to integer for window_size and num_units
    deep_size_bits = BitArray(ga_individual_solution[0:6])
    num_units_bits = BitArray(ga_individual_solution[6:10])
    learning_rate_bits = BitArray(ga_individual_solution[10:])
    
    deep_size = deep_size_bits.uint +1
    num_units = num_units_bits.uint +1
    learning_rate = (learning_rate_bits.uint+1)*10**(-4)
    
    print('\nDeep Size: ', deep_size, ', Num of Units: ', num_units, ', Learning rate: ', learning_rate)
    
    # Return fitness score of 100 if window_size or num_unit is zero
    if deep_size == 0 or num_units == 0 or learning_rate == 0:
        return 10000, 
    
    # Segment the train_data based on new window_size; split into train and validation (80/20)
    X,Y = prepare_dataset(train_data)
    X_train, X_val, y_train, y_val = split(X, Y, test_size = 0.20, random_state = 1120)
    
    # Train LSTM model and predict on validation set
    model = keras.Sequential()
    model.add(keras.Input(shape=(3,)))
    model.add(layers.Dense(num_units, input_shape=(3,)))
#     x = LSTM(num_units, input_shape=(window_size,1))(inputs)
    
    for i in range(deep_size):        
        model.add(layers.Dense(num_units, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(optimizer=optimizer,loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=5, batch_size=10, shuffle=True)
    y_pred = model.predict(X_val)
    
    # Calculate the RMSE score as fitness score for GA
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    print('Validation RMSE: ', rmse,'\n')
    
    return rmse,

A continuación, use la paquetería DEAP para definir las cosas para ejecutar GA. Usaremos una representación binaria para la solución de longitud diez. Se inicializará aleatoriamente utilizando la distribución de Bernoulli. Del mismo modo, se utiliza el crossover ordenado, la mutación aleatoria y la selección de la rueda de la ruleta. Los valores del parámetro GA se inicializan arbitrariamente; Te sugerimos que juegues con diferentes configuraciones.



In [16]:
population_size = 4
num_generations = 3
gene_length = 14

# As we are trying to minimize the RMSE score, that's why using -1.0. 
# In case, when you want to maximize accuracy for instance, use 1.0
creator.create('FitnessMax', base.Fitness, weights = (-1.0,))
creator.create('Individual', list , fitness = creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('binary', bernoulli.rvs, 0.5)
toolbox.register('individual', tools.initRepeat, creator.Individual, toolbox.binary, 
n = gene_length)
toolbox.register('population', tools.initRepeat, list , toolbox.individual)

toolbox.register('mate', tools.cxOrdered)
toolbox.register('mutate', tools.mutShuffleIndexes, indpb = 0.6)
toolbox.register('select', tools.selRoulette)
toolbox.register('evaluate', train_evaluate)

population = toolbox.population(n = population_size)
r = algorithms.eaSimple(population, toolbox, cxpb = 0.4, mutpb = 0.1, 
                        ngen = num_generations, verbose = True)

C:\Users\Joshu\.conda\envs\tensorflow\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\Joshu\.conda\envs\tensorflow\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)



Deep Size:  42 , Num of Units:  9 , Learning rate:  0.0009000000000000001
Train on 6444 samples
Epoch 1/5
6444/6444 [==============================] - ETA: 0s - loss: 0.0962- E - 18s 3ms/sample - loss: 0.0960
Epoch 2/5
6444/6444 [==============================] - 7s 1ms/sample - loss: 0.0021
Epoch 3/5
6444/6444 [==============================] - 7s 1ms/sample - loss: 0.0021
Epoch 4/5
6444/6444 [==============================] - 7s 1ms/sample - loss: 0.0021
Epoch 5/5
6444/6444 [==============================] - 8s 1ms/sample - loss: 0.0021ETA: 0s -
Validation RMSE:  0.04496596678259899 


Deep Size:  7 , Num of Units:  13 , Learning rate:  0.0015
Train on 6444 samples
Epoch 1/5
6444/6444 [==============================] - 7s 1ms/sample - loss: 12.0084
Epoch 2/5
6444/6444 [==============================] - 5s 712us/sample - loss: 1.2064
Epoch 3/5
6444/6444 [==============================] - 4s 624us/sample - loss: 0.0125
Epoch 4/5
6444/6444 [==============================] - 4s 551us/sa

La solución K mejor encontrada a través de GA se puede ver fácilmente usando tools.selBest(population,k = 1). Después, la configuración óptima se puede utilizar para entrenar en el conjunto de entrenamiento completo y probarlo en el conjunto de prueba de espera.

In [17]:
# Print top N solutions - (1st only, for now)
best_individuals = tools.selBest(population,k = 1)
best_deep_size = None
best_num_units = None
best_learning_rate = None

for bi in best_individuals:
    deep_size_bits = BitArray(bi[0:6])
    num_units_bits = BitArray(bi[6:10])
    learning_rate_bits = BitArray(bi[6:10])
        
    best_deep_size = deep_size_bits.uint +1
    best_num_units = num_units_bits.uint +1
    best_learning_rate = (learning_rate_bits.uint + 1)*10**(-4)
    print('\nDeep Size: ', best_deep_size, ', Num of Units: ', best_num_units, ', Learning rate: ', best_learning_rate)


Deep Size:  33 , Num of Units:  1 , Learning rate:  0.0001


In [18]:
# Train the model using best configuration on complete training set 
#and make predictions on the test set
X_train,y_train = prepare_dataset(train_data)
X_test, y_test = prepare_dataset(test_data)

model = keras.Sequential()
model.add(keras.Input(shape=(3,)))
model.add(layers.Dense(best_num_units, input_shape=(3,)))
#     x = LSTM(num_units, input_shape=(window_size,1))(inputs)

for i in range(best_deep_size):        
    model.add(layers.Dense(best_num_units, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

optimizer = keras.optimizers.Adam(learning_rate=best_learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
model.compile(optimizer=optimizer, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=5, batch_size=10,shuffle=True)
y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('Test RMSE: ', rmse)

Train on 8056 samples
Epoch 1/5
8056/8056 [==============================] - 19s 2ms/sample - loss: 1.0476
Epoch 2/5
8056/8056 [==============================] - 11s 1ms/sample - loss: 0.8943
Epoch 3/5
8056/8056 [==============================] - 12s 2ms/sample - loss: 0.7569
Epoch 4/5
8056/8056 [==============================] - 11s 1ms/sample - loss: 0.63360s - los
Epoch 5/5
8056/8056 [==============================] - 10s 1ms/sample - loss: 0.5233
Test RMSE:  0.7170667180173176


In [10]:
np.zeros(12)[10:]

array([0., 0.])